<a href="https://colab.research.google.com/github/HermanMoreno98/Centec_Python/blob/main/S01_5_Advanced_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorical Data

## Background and motivation

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
values = pd.Series(["apple","orange","apple","apple"]*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [ ]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [ ]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [ ]:
values = pd.Series([0,1,0,0]*2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [ ]:
dim = pd.Series(["apple","orange"])
dim

0     apple
1    orange
dtype: object

In [ ]:
#We can use the take method to restore the original Series of strings:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

## Categorical type in pandas

In [ ]:
fruits = ["apple","orange","apple","apple"]*2

In [ ]:
N = len(fruits)
N

8

In [ ]:
df = pd.DataFrame({
    "fruit":fruits,
    "basket_id":np.arange(N),
    "count":np.random.randint(3,15,size=N),
    "weight":np.random.uniform(0,4,size=N)
},columns=["basket_id","fruit","count","weight"])
df

,basket_id,fruit,count,weight
0,0,apple,12,0.980170
1,1,orange,5,3.912617
2,2,apple,4,2.726377
3,3,apple,10,0.416306
4,4,apple,4,1.988887
5,5,orange,11,2.931909
6,6,apple,3,1.090801
7,7,apple,11,2.446604


In [ ]:
#df['fruit'] is an array of Python string objects. We can convert it to categori‐ cal by calling:
fruit_cat = df["fruit"].astype("category")
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [ ]:
c = fruit_cat.values
c

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [ ]:
type(c)

pandas.core.arrays.categorical.Categorical

In [ ]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [ ]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [ ]:
#You can convert a DataFrame column to categorical by assigning the converted result:
df["fruit"] = df["fruit"].astype("category")
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [ ]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [ ]:
#If you have obtained categorical encoded data from another source, you can use the alternative from_codes constructor:
categories = ['foo', 'bar', 'baz']
codes=[0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [ ]:
ordered_cat = pd.Categorical.from_codes(codes, categories,ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [ ]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

## Computations with Categoricals

In [ ]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [ ]:
bins = pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] <
                                           (0.63, 3.928]]

In [ ]:
bins = pd.qcut(draws,4,labels=["Q1","Q2","Q3","Q4"])
bins
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [ ]:
#The labeled bins categorical does not contain information about the bin edges in the data, so we can use groupby to extract some summary statistics:
bins = pd.Series(bins,name="quartile")
bins

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
       ..
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, Length: 1000, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [ ]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(["count","min","max"])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [ ]:
results["quartile"]

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

## Categorical Methods

In [ ]:
s = pd.Series(["a","b","c","d"]*2)
s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: object

In [ ]:
cat_s = s.astype("category")
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [ ]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
actual_categories = ['a', 'b', 'c', 'd','e']

In [ ]:
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [ ]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [ ]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [ ]:
# Creating dummy varibales for modelling
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# Advanced GroupBy Use

In [ ]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [ ]:
g = df.groupby("key").value

In [ ]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [ ]:
#Suppose instead we wanted to produce a Series of the same shape as df['value'] but with values replaced by the average grouped by 'key'
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [ ]:
#As a more complicated example, we can compute the ranks in descending order for each group:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [ ]:
#Consider a group transformation function composed from simple aggregations:
def normalize(x):
  return (x - x.mean()) / x.std()

In [ ]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
g.apply(normalize)

<ipython-input-68-f5e6a374d729>:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  g.apply(normalize)


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64